# Vectors
In this tutorial we refresh some knowledge on vectors.

In meteorology and climatology we often work with variables that have four dimensions: $x$, $y$, $z$ and $t$, which represent the variable in the three-dimensional space and in time. These variables can be split into two categories: scalars and vectors. Scalars, such as temperature, humidity and pressure do not have a direction, whereas vectors, such as wind or the temperature gradient do. In this tutorial, we refresh the basics of vectors, their notation and related rules and mathematical operations.

We express vectors using bold letters: $\boldsymbol{u}$ is the wind vector. This vector is defined as:

$$\boldsymbol{u} \equiv \left( u, v, w\right).$$

Its three components are thus the east-west wind $u$, the north-south wind $v$ and the vertical wind $w$.

In [ ]:
from ipywidgets import interact # Package ipywidgets contains the features of interactive notebooks.
import numpy as np              # Numpy is the fundamental package for scientific computing in Python.
import netCDF4 as nc            # NetCDF is the data format of the meteorological data that we use.
import matplotlib.pyplot as pl  # Matplotlib is a scientific plotting package.
# The statement below enforces the plots to be put into this notebook, instead of in their own windows.
%pylab inline
pl.rcParams.update({'font.size': 12})          # Set the standard font size of the plots to 11pt.
pl.rcParams.update({'figure.figsize': [12,6]}) # Set the standard figure size.

In [ ]:
nc_file = nc.Dataset("dyn_met.nc", "r")
lat = nc_file.variables["lat"][:]
lon = nc_file.variables["lon"][:]
u   = nc_file.variables["ua" ][-1,0,:,:]
v   = nc_file.variables["va" ][-1,0,:,:]
zg  = nc_file.variables["zg" ][-1,0,:,:]

Calculate a scalar $u_{tot}$ that is the magnitude of the wind vector:

In [ ]:
# Write the correct Python code below, replace the random numbers with the correct code.
# u_tot = np.random.rand(lat.size, lon.size)
u_tot = (u**2 + v**2)**.5

In [ ]:
pl.contourf(lon, lat, u_tot, 16, cmap=pl.cm.viridis)
pl.colorbar()
pl.xlim(lon.min(), lon.max())
pl.ylim(lat.min(), lat.max())
pl.xlabel('lat')
pl.ylabel('lon');

___

In [ ]:
def plot_p_u_vector(lon_min=lon.min(), lon_max=lon.max(), lat_min=lat.min(), lat_max=lat.max()):
    pl.contourf(lon, lat, u_tot, 20, cmap=pl.cm.YlOrRd)
    pl.colorbar()
    pl.streamplot(lon, lat, u, v, density=5, color='k')
    pl.xlim(lon_min, lon_max)
    pl.ylim(lat_min, lat_max)
    pl.xlabel('lon')
    pl.ylabel('lat')
    pl.savefig('circulation.pdf');

interact(plot_p_u_vector, lon_min=(0, 360), lon_max=(0,360), lat_min=(-90, 90), lat_max=(-90, 90), __manual = True);

The plot above shows the wind speed:

$$ u = 10 $$

In [ ]:
step = 2
lon_min = 180
lon_max = 300
lat_min =   0
lat_max =  80
ilon_min = (abs(lon-lon_min)).argmin()
ilon_max = (abs(lon-lon_max)).argmin()
ilat_min = (abs(lat-lat_min)).argmin()
ilat_max = (abs(lat-lat_max)).argmin()
ilat_min, ilat_max = ilat_max, ilat_min

pl.contourf(lon[ilon_min:ilon_max+1],
            lat[ilat_min:ilat_max+1],
            zg[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
            20, cmap=pl.cm.viridis)
pl.colorbar()

pl.streamplot(lon[ilon_min:ilon_max+1],
              lat[ilat_min:ilat_max+1],
              u[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
              v[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
              density=4, color='k')

#pl.quiver(lon[ilon_min:ilon_max+1:step],
#          lat[ilat_min:ilat_max+1:step],
#          u[ilat_min:ilat_max+1:step, ilon_min:ilon_max+1:step],
#          v[ilat_min:ilat_max+1:step, ilon_min:ilon_max+1:step],
#          pivot='mid')

pl.xlim(lon_min, lon_max)
pl.ylim(lat_min, lat_max)
pl.xlabel('lon')
pl.ylabel('lat');

The divergence of a vector is a scalar. In vector notation, we write it down as:
$$\text{div} \left( \boldsymbol{u} \right) \equiv \nabla \cdot \boldsymbol{u}$$
Expand the right hand side of the previous expression:

$$\nabla \cdot \boldsymbol{u} = \dots$$
$$\nabla \cdot \boldsymbol{u} = \dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} $$

In [ ]:
r_earth = 6.37e6
dx = 2.*np.pi*r_earth*cos(np.pi/180*lat) / lon.size
dy = np.pi*r_earth / lat.size
divu = np.zeros(u.shape)
for l in range(lat.size):
    divu[l,:] += np.gradient(u[l,:], dx[l])
for l in range(lon.size):
    divu[:,l] += np.gradient(v[:,l], dy)

In [ ]:
divu_max = (abs(divu[ilat_min:ilat_max+1, ilon_min:ilon_max+1]).max())
divu_range = np.linspace(-divu_max, divu_max, 20)
pl.contourf(lon[ilon_min:ilon_max+1],
            lat[ilat_min:ilat_max+1],
            divu[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
            divu_range, cmap=pl.cm.RdBu, extend='both')
pl.colorbar()

pl.streamplot(lon[ilon_min:ilon_max+1],
              lat[ilat_min:ilat_max+1],
              u[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
              v[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
              density=4, color='k')

pl.xlim(lon_min, lon_max)
pl.ylim(lat_min, lat_max)
pl.xlabel('lon')
pl.ylabel('lat');

In [ ]:
dlon = 2.*np.pi*cos(np.pi/180*lat) / lon.size
dlat = np.pi / lat.size
divu_sphere = np.zeros(u.shape)
cos_lat = np.cos(np.pi/180.*lat)
for l in range(lat.size):
    divu_sphere[l,:] += np.gradient(u[l,:], dlon[l]) / (r_earth*cos_lat[l])
for l in range(lon.size):
    divu_sphere[:,l] += np.gradient(cos_lat*v[:,l], dlat) / (r_earth*cos_lat)

In [ ]:
divu_sphere_max = (abs(divu_sphere[ilat_min:ilat_max+1, ilon_min:ilon_max+1]).max())
divu_sphere_range = np.linspace(-divu_sphere_max, divu_sphere_max, 20)
pl.contourf(lon[ilon_min:ilon_max+1],
            lat[ilat_min:ilat_max+1],
            divu_sphere[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
            divu_sphere_range, cmap=pl.cm.RdBu, extend='both')
pl.colorbar()

pl.streamplot(lon[ilon_min:ilon_max+1],
              lat[ilat_min:ilat_max+1],
              u[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
              v[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
              density=4, color='k')

pl.xlim(lon_min, lon_max)
pl.ylim(lat_min, lat_max)
pl.xlabel('lon')
pl.ylabel('lat');

In [ ]:
pl.contourf(lon[ilon_min:ilon_max+1],
            lat[ilat_min:ilat_max+1],
            divu       [ilat_min:ilat_max+1, ilon_min:ilon_max+1]-
            divu_sphere[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
            divu_sphere_range, cmap=pl.cm.RdBu, extend='both')
pl.colorbar()

pl.streamplot(lon[ilon_min:ilon_max+1],
              lat[ilat_min:ilat_max+1],
              u[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
              v[ilat_min:ilat_max+1, ilon_min:ilon_max+1],
              density=4, color='k')

pl.xlim(lon_min, lon_max)
pl.ylim(lat_min, lat_max)
pl.xlabel('lon')
pl.ylabel('lat');